In [1]:
# files and system
import os
import sys

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# working with images
import cv2
import imageio as iio
from PIL import Image
import scipy.ndimage
import skimage.transform

In [4]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

import torchsummary

# Global Variables

In [5]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # select device for training, i.e. gpu or cpu
print(DEVICE)

cpu


# EDA

## Mask channels

In [6]:
img = iio.v3.imread('../data/train-val/images/cju5buy2bal250818ipl6fqwv.jpg')
img.shape

(430, 569, 3)

In [7]:
mask = iio.v3.imread('../data/train-val/masks/cju5buy2bal250818ipl6fqwv.jpg')
mask.shape

(430, 569, 3)

In [8]:
print((mask[:, :, 0] == mask[:, :, 1]).all())
print((mask[:, :, 1] == mask[:, :, 2]).all())

True
True


**Will only use the 0-th channel for all masks.**

In [9]:
mask = iio.v3.imread('../data/train-val/masks/cju5buy2bal250818ipl6fqwv.jpg')[:, :, 0]
mask.shape

(430, 569)

## Mask values

In [11]:
mask = iio.v3.imread('../data/train-val/masks/cju5buy2bal250818ipl6fqwv.jpg')[:, :, 0]
np.unique(mask)

array([  0,   1,   2,   3,   4,   5,   6,   7, 248, 249, 250, 251, 252,
       253, 254, 255], dtype=uint8)

In [12]:
mask = iio.v3.imread('../data/train-val/masks/cju5buy2bal250818ipl6fqwv.jpg')[:, :, 0]
mask = (mask / 255).round()
np.unique(mask)

array([0., 1.])

## Pre-processing flow

In [13]:
# change intensity [0, 255] -> [0, 1]  and binarize masks
img = iio.v3.imread('../data/train-val/images/cju5buy2bal250818ipl6fqwv.jpg') / 255
mask = iio.v3.imread('../data/train-val/masks/cju5buy2bal250818ipl6fqwv.jpg')[:, :, 0] / 255
mask = mask.round()

In [14]:
img = torch.FloatTensor(np.transpose(img, [2, 0 ,1])) # make the image channel-first
mask = torch.FloatTensor(mask).unsqueeze(0)

In [15]:
img.shape, mask.shape

(torch.Size([3, 430, 569]), torch.Size([1, 430, 569]))

In [16]:
# this way same transformation can be applied 
sample = torch.cat((img, mask), 0)
sample.shape

torch.Size([4, 430, 569])

## Pre-defined split

In [17]:
os.listdir('../train-val-split')

['train.txt', 'README.md', 'val.txt']

In [18]:
with open('../train-val-split/train.txt', 'r') as f:
    ids_train = [l.strip()+'.jpg' for l in f]
with open('../train-val-split/val.txt', 'r') as f:
    ids_val = [l.strip()+'.jpg' for l in f]

In [19]:
print(len(ids_train), len(ids_val))

880 120
